<a href="https://colab.research.google.com/github/dr-kinder/playground/blob/master/dielectric-rod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dielectric Rod

This notebook models a dielectric rod in a uniform electric field.  It also introduces a method for constructing models in GMSH that have multiple components.

The model lets us explore a range of systems.
- Set `epsilon1` to 1 to see the potential and fields in vacuum.
- Set `epsilon1` from 2–5 to see the fields of a dielectric rod.
- Set `epsilon1` to a value greater than 100 to see a conducting sphere — approximately.

# Install and Import

In [1]:
try:
    # Import gmsh library for generating meshes.
    import gmsh
except ImportError:
    # If it is not available, install it.  Then import it.
    !wget "https://fem-on-colab.github.io/releases/gmsh-install.sh" -O "/tmp/gmsh-install.sh" && bash "/tmp/gmsh-install.sh"
    import gmsh

In [2]:
try:
    # Import FEniCSx libraries for finite element analysis.
    import dolfinx
except ImportError:
    # If they are not found, install them.  Then import them.
    !wget "https://fem-on-colab.github.io/releases/fenicsx-install-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
    import dolfinx

In [3]:
try:
    # Import multiphenicsx, mainly for plotting.
    import multiphenicsx
except ImportError:
    # If they are not found, install them.
    !pip3 install "multiphenicsx@git+https://github.com/multiphenics/multiphenicsx.git@8b97b4e"
    import multiphenicsx

In [4]:
# Everything should be installed now.
# Import the rest of what we need.

import dolfinx.fem
import dolfinx.io
import gmsh
import mpi4py.MPI
import numpy as np
import petsc4py.PETSc
import ufl
import multiphenicsx.io

# Build a model with GMSH

This model has two separate parts that we want to combine into a single mesh.  GMSH won't complain if surfaces overlap, but FEniCSx will.  There are several ways to define a single mesh with multiple entities using GMSH.  (See [the GMSH tutorials](https://gitlab.onelab.info/gmsh/gmsh/-/tree/gmsh_4_9_5/tutorials/python) for some examples — in particular, `t4.py` and `t5.py`.)

This example creates a stencil.  We define the "universe" to be the entire region of interest — a large rectangle.  The "universe" is actually two distinct regions in this system: a dielectric rod and the background.  We define the boundaries of the rectangle and rod.  Then we define the background to be the rectangle, but with a hole cut out for the rod.  And then we define a separate surface for the rod.  We then introduce the physical objects: the boundaries, the background, and the rod.  They must fit together like pieces of a jigsaw puzzle.

Plotting the mesh shows the separate regions.  They are tagged in the model, too, so we can assign different properties to each, as is done later in this notebook.

In [81]:
# Define the dimension of the model.
dim = 2

# Define material properties.
epsilon0 = 1
epsilon1 = 10

# Define the center of the universe.
x0 = 0
y0 = 0
z0 = 0

# Define the rectangle.
L = 4
W = 4

# Define the circle.
rX = 1
rY = 1

# Grid size parameter.  Make it smaller for higher resolution.
delta = 0.1

In [82]:
# Create the model.
gmsh.initialize()
gmsh.model.add("mesh")

## Define rectangle.
# Corners
p0 = gmsh.model.geo.addPoint(x0-L/2, y0-W/2, z0, delta)
p1 = gmsh.model.geo.addPoint(x0+L/2, y0-W/2, z0, delta)
p2 = gmsh.model.geo.addPoint(x0+L/2, y0+W/2, z0, delta)
p3 = gmsh.model.geo.addPoint(x0-L/2, y0+W/2, z0, delta)

# Edges
l0 = gmsh.model.geo.addLine(p0, p1)
l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p0)
perimeter = gmsh.model.geo.addCurveLoop([l0, l1, l2, l3])

# Define points: center of circle and two points on opposite sides.
c0 = gmsh.model.geo.addPoint(x0, y0, z0, delta)
c1 = gmsh.model.geo.addPoint(x0, y0-rY, z0, delta)
c2 = gmsh.model.geo.addPoint(x0, y0+rY, z0, delta)

# Define two semicircular arcs that will be joined into a circle.
s0 = gmsh.model.geo.addCircleArc(c1, c0, c2)
s1 = gmsh.model.geo.addCircleArc(c2, c0, c1)
circumference = gmsh.model.geo.addCurveLoop([s0, s1])

# The "background" is a rectangle with a hole.
rectangle = gmsh.model.geo.addPlaneSurface([perimeter, circumference])

# The object fills the hole.
circle = gmsh.model.geo.addPlaneSurface([circumference])

# Update the model with all of the features we add.
gmsh.model.geo.synchronize()

# Identify the physical objects: boundaries and surfaces.
# Add the edges separately, so boundary conditions can be set individually.
gmsh.model.addPhysicalGroup(1, [l0], 1)
gmsh.model.addPhysicalGroup(1, [l1], 2)
gmsh.model.addPhysicalGroup(1, [l2], 3)
gmsh.model.addPhysicalGroup(1, [l3], 4)
gmsh.model.addPhysicalGroup(2, [rectangle], 1)

# Add the circle and its boundaries.
gmsh.model.addPhysicalGroup(1, [c0,c1], 5)
gmsh.model.addPhysicalGroup(2, [circle], 5)

# Create a mesh for this system.
gmsh.model.mesh.generate(dim)

# Bring the mesh into FEniCSx.
mesh, subdomains, boundaries = dolfinx.io.gmshio.model_to_mesh(
    gmsh.model, comm=mpi4py.MPI.COMM_WORLD, rank=0, gdim=2)

# Close the mesh generating program.
gmsh.finalize()

In [83]:
# Plot the entire mesh.
multiphenicsx.io.plot_mesh(mesh)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [84]:
# Plot the subdomains that FEniCSx has identified.
multiphenicsx.io.plot_mesh_tags(subdomains)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [85]:
# Inspect the boundaries of the elements and the system.
multiphenicsx.io.plot_mesh_tags(boundaries)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [86]:
# Set the potential on each wall.
# The two ends will be fixed.  The two sides will be open.
V1 = 2.0
V2 = 0.0
V3 = 0.0
V4 = 0.0

# Define the type of boundary on each wall.
# Set to "True" or "1" for essential (fixed potential).
# Set to "False" or "0" for natural (zero normal derivative).
essential_1 = True
essential_2 = False
essential_3 = True
essential_4 = False

In [87]:
# Set the relative permittivity.

# Define a simple function space for properties.
Q = dolfinx.fem.FunctionSpace(mesh, ("DG", 0))

# Get the catalog of materials.
material_tags = np.unique(subdomains.values)

# Define function for permittivity.
epsilon = dolfinx.fem.Function(Q)

# Now, cycle over all objects and assign material properties. 
for tag in material_tags:
    cells = subdomains.find(tag)
    
    # Set values for magnetic permeability.
    if tag == 1:
        # Vacuum
        eps_ = epsilon0
    elif tag == 5:
        # Dielectric
        eps_ = epsilon1
    else:
        # Something else
        eps_ = epsilon0

    epsilon.x.array[cells] = np.full_like(cells, eps_, dtype=petsc4py.PETSc.ScalarType)

In [88]:
## Set up the finite element problem.

# Define trial and test functions.
V = dolfinx.fem.FunctionSpace(mesh, ("Lagrange", 2))

# Define the trial and test functions.
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# Create a function to store the solution.
phi = dolfinx.fem.Function(V)

# Identify the domain (all the points inside the boundary).
Omega = subdomains.indices[subdomains.values == 1]

# Identify the boundary for FEniCSx.
wall_1 = boundaries.indices[boundaries.values == 1]
wall_2 = boundaries.indices[boundaries.values == 2]
wall_3 = boundaries.indices[boundaries.values == 3]
wall_4 = boundaries.indices[boundaries.values == 4]
dOmega_1 = dolfinx.fem.locate_dofs_topological(V, boundaries.dim, wall_1)
dOmega_2 = dolfinx.fem.locate_dofs_topological(V, boundaries.dim, wall_2)
dOmega_3 = dolfinx.fem.locate_dofs_topological(V, boundaries.dim, wall_3)
dOmega_4 = dolfinx.fem.locate_dofs_topological(V, boundaries.dim, wall_4)

# Now introduce the boundary conditions.
# Store the essential boundary conditions in a list.
essential_bc = []
if essential_1:
    Phi0 = dolfinx.fem.Constant(mesh, petsc4py.PETSc.ScalarType(V1))
    essential_bc += [dolfinx.fem.dirichletbc(Phi0, dOmega_1, V)]
if essential_2:
    Phi0 = dolfinx.fem.Constant(mesh, petsc4py.PETSc.ScalarType(V2))
    essential_bc += [dolfinx.fem.dirichletbc(Phi0, dOmega_2, V)]
if essential_3:
    Phi0 = dolfinx.fem.Constant(mesh, petsc4py.PETSc.ScalarType(V3))
    essential_bc += [dolfinx.fem.dirichletbc(Phi0, dOmega_3, V)]
if essential_4:
    Phi0 = dolfinx.fem.Constant(mesh, petsc4py.PETSc.ScalarType(V4))
    essential_bc += [dolfinx.fem.dirichletbc(Phi0, dOmega_4, V)]

# This is the FEM version of the Laplacian.
# It is the left-hand side of Poisson's or Laplace's equation.
a = epsilon * ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx

# This the right-hand side of Poisson's equation.
# We need to create a FEniCSx-friendly version of 0.
Zero = dolfinx.fem.Constant(mesh, petsc4py.PETSc.ScalarType(0.0))
L = Zero * v * ufl.dx

# Put it all together for FEniCSx.
problem = dolfinx.fem.petsc.LinearProblem(a, L, essential_bc, u=phi)

# Now, solve it!
problem.solve()

# Tie up some loose ends.
phi.vector.ghostUpdate(addv=petsc4py.PETSc.InsertMode.INSERT, mode=petsc4py.PETSc.ScatterMode.FORWARD)

In [89]:
# Plot the potential.
multiphenicsx.io.plot_scalar_field(phi, "Potential", warp_factor=1)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [90]:
# Define a set of elements for a vector field.
W = dolfinx.fem.VectorFunctionSpace(mesh, ("Lagrange", 2))
E = dolfinx.fem.Function(W)

# Compute the gradient as a symbolic expression, then interpolate it onto the mesh.
expr = dolfinx.fem.Expression(ufl.as_vector((-phi.dx(0), -phi.dx(1))), W.element.interpolation_points())
E.interpolate(expr)

In [91]:
# Use multiphenics to plot the vector field.
multiphenicsx.io.plot_vector_field(E,name="Electric Field", glyph_factor=0.1)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…